In [1]:
import datetime
import pandas as pd
import pickle

In [2]:
model = pickle.load(open("model.pkl", "rb"))
columns_model_only = pickle.load(open("columns_model_only.pkl", "rb"))

## Generate Forecasts

In [3]:
df_test = pd.read_csv("data/test_clean.csv", header=0, index_col=0)
X_test = df_test[columns_model_only]

In [4]:
price_forecasts = model.predict(X_test)

In [5]:
test_ids = pd.read_csv("data/test.csv", header=0)["Id"]

In [18]:
df_output = pd.DataFrame(columns=["Id", "SalePrice"])
for i in range(len(price_forecasts)):
    df_output.loc[len(df_output)] = [int(test_ids[i]), price_forecasts[i]]
df_output["Id"] = pd.Series(df_output["Id"], dtype=int) # required format of Id column is int

# Remove non-zero results
df_output.loc[0, "SalePrice"] = -1000
min_positive_price = min(df_output.loc[df_output["SalePrice"]>0, "SalePrice"])
df_output.loc[df_output["SalePrice"]<0, "SalePrice"] = min_positive_price

In [7]:
df_output.to_csv("submission/submission_{}.csv".format(datetime.datetime.now()), header=True, index=False)